In [ ]:
import pandas as pd
from sqlalchemy import create_engine

user = 'project'
password = 'myproject'
host = 'localhost'
database = 'stat-uz'

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

csv_file_path = 'main_df.csv'
df = pd.read_csv(csv_file_path)

df.to_sql('uz-data', con=engine, if_exists='replace', index=False)

In [3]:
import os
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# MySQLga ulanish uchun sozlamalar
def mysql_ulanis(db_host, db_user, db_password, db_name):
    try:
        # SQLAlchemy motorini yaratish
        engine = create_engine(f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}")
        return engine
    except Exception as e:
        print(f"MySQL ulanishida xatolik: {e}")
        return None

# CSV fayllarni o'qish va ularni MySQL ga kiritish
def csvdan_mysqlga(folder_path, engine):
    try:
        # Papkadagi barcha fayllarni olish
        fayllar = os.listdir(folder_path)
        # Faqat .csv fayllarni tanlab olish
        csv_fayllar = [fayl for fayl in fayllar if fayl.endswith('.csv')]
        
        for fayl in csv_fayllar:
            file_path = os.path.join(folder_path, fayl)
            # Fayl nomidan jadval nomini olish (".csv" qismini olib tashlash)
            table_name = os.path.splitext(fayl)[0]
            
            # CSV faylni Pandas yordamida o'qish
            df = pd.read_csv(file_path)
            
            # Jadvalni MySQL ga yozish
            df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
            print(f"Fayl '{fayl}' jadvalga '{table_name}' nomi bilan saqlandi.")
    except Exception as e:
        print(f"Xatolik: {e}")

# Foydalanish
if __name__ == "__main__":
    folder_path = '/home/uznetdev/Desktop/project/AdventureWorks Dashboard/sql-server-samples-master/samples/databases/adventure-works/data-warehouse-install-script'
    
    # MySQL ulanish sozlamalari
    db_host = 'localhost'        # Yoki server IP manzili
    db_user = 'project'             # MySQL foydalanuvchi nomi
    db_password = 'myproject'        # MySQL paroli
    db_name = 'adventure-works'    # MySQL database nomi

    engine = mysql_ulanis(db_host, db_user, db_password, db_name)
    
    if engine:
        csvdan_mysqlga(folder_path, engine)


ModuleNotFoundError: No module named 'pandas'